In [1]:
import sys
from pathlib import Path

# Add src/ to the Python path
sys.path.append(str(Path().resolve() / "src"))
sys.path.append(str(Path().resolve() / "data"))

<h1>Interactive Widgets</h1>

An assortment of widgets to adjust/calculate the odds of a particular DV check. Currently uses code from dv_check.py primarily.


In [2]:
from ipywidgets import interact

from cyberpunk.dice import roll_dice
from cyberpunk.check import Check

def general_skill_check(BASE, MODIFIERS, DV, SHOW_ALL):
    possibilities = roll_dice(dice=[-10], modifiers=[MODIFIERS+BASE])
    my_check = Check(rolls=[possibilities], dv=DV)

    print(my_check.full_stats(use_thresholds=(not SHOW_ALL)))

print("General Skill Check")
interact(general_skill_check, BASE=(0,20), MODIFIERS=(-10,10), DV=(0,40), SHOW_ALL=False)

General Skill Check


interactive(children=(IntSlider(value=10, description='BASE', max=20), IntSlider(value=0, description='MODIFIE…

<function __main__.general_skill_check(BASE, MODIFIERS, DV, SHOW_ALL)>

In [3]:
def interface_check(INTERFACE, MODIFIERS, DV, SHOW_ALL):
    return general_skill_check(INTERFACE, MODIFIERS, DV, SHOW_ALL)

print("Interface Check")
interact(interface_check, INTERFACE=(0,20), MODIFIERS=(-10,10), DV=(0,20), SHOW_ALL=False)

Interface Check


interactive(children=(IntSlider(value=10, description='INTERFACE', max=20), IntSlider(value=0, description='MO…

<function __main__.interface_check(INTERFACE, MODIFIERS, DV, SHOW_ALL)>

In [4]:
def contested_skill_check(BASE, MODIFIERS, DV, OPPONENT, OPPONENT_ROLLS, SHOW_ALL):

        my_check = None
        if OPPONENT_ROLLS:
            my_roll = roll_dice(dice=[-10], modifiers=[MODIFIERS+BASE])
            opponent_roll = roll_dice(dice=[10], modifiers=[OPPONENT]) * -1
            my_check = Check(rolls=[my_roll, opponent_roll], dv=DV)
        else:
            my_roll = roll_dice(dice=[-10], modifiers=[MODIFIERS+BASE-OPPONENT])
            my_check = Check(rolls=[my_roll], dv=DV)

        print(my_check.full_stats(use_thresholds=(not SHOW_ALL)))

print("Skill Check Against Opponent")
interact(contested_skill_check, BASE=(0,20), MODIFIERS=(-10,10), DV=(0,40), OPPONENT=(0,20), OPPONENT_ROLLS=False, SHOW_ALL=False)

Skill Check Against Opponent


interactive(children=(IntSlider(value=10, description='BASE', max=20), IntSlider(value=0, description='MODIFIE…

<function __main__.contested_skill_check(BASE, MODIFIERS, DV, OPPONENT, OPPONENT_ROLLS, SHOW_ALL)>

In [5]:
def net_damage_against_attack(HP, INTERFACE, PRG_ATK, ATK_ROLLS, SHOW_ALL):


    # Hit Chance
    my_evasion = roll_dice([-10], modifiers=[int(INTERFACE)]) * -1
    opponent_hit = roll_dice(dice=[-10], modifiers=[int(PRG_ATK)])

    hit_check = Check(rolls=[opponent_hit, my_evasion])
    print(f"Accuracy\n{str(hit_check)}")

    if ATK_ROLLS != "n/a":
        # Damage if the hit connectss
        
        times, die = str(ATK_ROLLS).split('d')

        opponent_rolls = [int(die)]*int(times)
        incoming_attack = roll_dice(dice=opponent_rolls, modifiers=[])

        my_check = Check(rolls=[incoming_attack], dv=0, ties_count=False)

        my_check.apply_accuracy(hit_check.success_chance)

        print(my_check.roll_results)
        print(my_check.full_stats(use_thresholds=(not SHOW_ALL), key_values=[0, times, HP//4, HP//2, HP]))

print("Attack Damage" \
"")
interact(net_damage_against_attack, HP=(0,50), INTERFACE=(0, 10), PRG_ATK=(0,30), ATK_ROLLS=['n/a','1d6','2d6','3d6','4d6','5d6','6d6'], SHOW_ALL=False)


Attack Damage


interactive(children=(IntSlider(value=25, description='HP', max=50), IntSlider(value=5, description='INTERFACE…

<function __main__.net_damage_against_attack(HP, INTERFACE, PRG_ATK, ATK_ROLLS, SHOW_ALL)>

In [6]:
import numpy as np
from cyberpunk.dice import roll_dice
import math
from collections import defaultdict
from ipywidgets import interact


def downtime_hustle_value(MAX_RANK, SHOW_ALL):

    #Rank Info
    low_rank = np.array([375, 275, 125, 125, 75, 175, 75, 125, 125, 275, 375])
    mid_rank = np.array([425, 325, 175, 175, 125, 225, 125, 175, 175, 325, 425])
    high_rank = np.array([475, 375, 225, 225, 175, 275, 175, 225, 225, 375, 475])

    if MAX_RANK == "1-4":
        rank = low_rank
    elif MAX_RANK == "5-7":
        rank = mid_rank
    else: #8-10
        rank = high_rank

    m = roll_dice([6, 6], [])

    _, counts = np.unique(m, return_counts=True)
    probs = counts / counts.sum()

    total_value = probs * rank
    print(f"Average Payout: {math.floor(total_value.sum()+.5)}eb\n")

    if SHOW_ALL:
        payouts = defaultdict(float)
        for eb_value, odds in zip(rank, probs):
            payouts[eb_value] += odds

        desc_payouts = sorted(payouts, reverse=True)

        running_odds = 0.0
        for eb_value in desc_payouts:
            running_odds += payouts[eb_value]
            print(f"≥{eb_value}eb\t {running_odds:.2%}")
        
interact(downtime_hustle_value, MAX_RANK=["1-4","5-7","8-10"], SHOW_ALL=True)



interactive(children=(Dropdown(description='MAX_RANK', options=('1-4', '5-7', '8-10'), value='1-4'), Checkbox(…

<function __main__.downtime_hustle_value(MAX_RANK, SHOW_ALL)>